尝试不用GEO信息

In [1]:
# %matplotlib inline
import torch
import torch.nn as nn
# import torch.nn.functional as F
import numpy as np
import h5py as h5
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from sklearn import preprocessing
torch.__version__

'1.12.1'

In [2]:
class NN2(nn.Module):
    def __init__(self):
        super(NN2, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(5, 128),
            nn.Sigmoid(),
            nn.Linear(128, 1),
        )

    def forward(self, x):
        logits_ = self.linear_relu_stack(x)
        logits_.detach()
        return logits_

In [3]:
def generate_data(filename: str, p_trans: preprocessing.StandardScaler):
    data = h5.File(filename, "r")
    pe_truth = data["PETruth"]
    sim_truth = data["ParticleTruth"]
    pe_id = pd.DataFrame({'id': pe_truth['EventID'], 'PMT': pe_truth['ChannelID'], 't': pe_truth['PETime']})  # PE事件
    pe_id = pe_id[pe_id['PMT'] < 17612]  # 只选取前17612个PMT
    id_df = pd.DataFrame({'id': sim_truth['EventID'], 'p': sim_truth['p']}).sort_values('id')
    p_array = id_df['p'].to_numpy()

    del pe_truth, sim_truth, data, id_df

    p_ts = torch.from_numpy(p_trans.fit_transform(p_array.reshape(-1, 1))).float()

    pe_id = pd.merge(pe_id, pe_id.groupby(['id', 'PMT'], as_index=False).size(), how='left')

    n_all = pe_id.groupby('id').size()
    in_mean = pe_id.groupby('id').mean()
    in_mean['PMT'] = n_all

    in_std = pe_id.groupby('id').std()

    in_df = pd.merge(in_mean, in_std, on='id', suffixes=('_mean', '_std'))
    in_df.drop(columns=['PMT_std'], inplace=True)

    in_ts = torch.tensor(in_df.to_numpy()).float()
    ln = nn.LayerNorm(in_ts.shape[0])
    in_ts = ln(in_ts.T).T
    in_ts = torch.tensor(in_ts.data.numpy()).float()

    del in_df, in_std, in_mean, n_all
    return in_ts, p_ts, p_trans

In [4]:
def training(in_ts: torch.Tensor, p_ts: torch.Tensor, batch_n=10000, step=5000, printing_step=500, filename_pt='nn2.pt'):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    nn2 = NN2().to(device)
    loss_func = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(nn2.parameters(), lr=0.002)
    for i in tqdm(range(step)):
        prediction_p = nn2(in_ts[:batch_n])
        loss = loss_func(prediction_p, p_ts[:batch_n])
        if not i % printing_step :
            print(loss)
        optimizer.zero_grad()
        loss.backward()  # retain_graph=True
        optimizer.step()
        if loss.data < 0.001 and i > 200:
            break
    torch.save(nn2.state_dict(), filename_pt)
    return nn2, loss_func

In [5]:
p_trans0 = preprocessing.StandardScaler()
in_ts0, p_ts0, p_trans0 = generate_data('0-exact.h5', p_trans0)

In [7]:
p_trans2 = preprocessing.StandardScaler()
in_ts2, p_ts2, p_trans2 = generate_data('2-exact.h5', p_trans2)

In [ ]:
p_trans3 = preprocessing.StandardScaler()
in_ts3, p_ts3, p_trans3 = generate_data('3-exact.h5', p_trans3)

In [6]:
nn2_0, loss_f = training(in_ts0, p_ts0, 10000, 5000)

  0%|          | 7/5000 [00:00<02:23, 34.70it/s]

tensor(1.2694, grad_fn=<MseLossBackward0>)


 20%|██        | 1012/5000 [00:18<01:15, 52.48it/s]

tensor(0.0045, grad_fn=<MseLossBackward0>)


 40%|████      | 2009/5000 [00:37<00:56, 52.49it/s]

...

 60%|██████    | 3008/5000 [00:55<00:38, 52.25it/s]

tensor(0.0030, grad_fn=<MseLossBackward0>)


 80%|████████  | 4005/5000 [01:15<00:19, 51.67it/s]

tensor(0.0027, grad_fn=<MseLossBackward0>)


100%|██████████| 5000/5000 [01:36<00:00, 51.90it/s]


In [8]:
loss_f(p_ts0, nn2_0(in_ts0))    # 训练集 '0-exact.h5'

tensor(0.0027, grad_fn=<MseLossBackward0>)

In [9]:
loss_f(p_ts2, nn2_0(in_ts2))    # 测试集 '2-exact.h5'

tensor(0.0042, grad_fn=<MseLossBackward0>)

In [10]:
def cal_error(net: nn.Module, input_data: np.ndarray, label: np.ndarray):
    return np.sqrt(np.mean((label.flatten() - net(torch.tensor(input_data)).data.numpy().flatten()) ** 2 / label.flatten()))

In [12]:
cal_error(nn2_0, in_ts0.data.numpy(), p_ts0.data.numpy())   # 训练集 '0-exact.h5'

0.012884793

In [11]:
cal_error(nn2_0, in_ts2.data.numpy(), p_ts2.data.numpy())   # 测试集 '2-exact.h5'

0.011503707